# Generate a coorindate / pixel list
This notebook will genereate three lists. This lists are used by the DeepRain App to calculate the pixel in the forecast image with longitude and latitude. 
### listLatitudeComplete
All latitude values in one list 
### listLongitudeComplete
All longitude values in one list 
### listCoordinates
For each pair of latitude / longitude the pixel 

Example: The 5th element in the latitude and longitude together list will give you something like [latitude, longitude]. The 5th element in listCoordinates will give you the pixel in the forecast of a 900x900 matrix. It will look like [y, x].

In [1]:
import wradlib as wrl
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pickle

In [6]:
radolan_grid_ll = wrl.georef.get_radolan_grid(1100,900, wgs84=True)
# The edges of the grid 
#corner up, right
print(radolan_grid_ll[1099,899, :])
#corner down, left
print(radolan_grid_ll[0, 0, :])
#corner up, left
print(radolan_grid_ll[1099, 0, :])
#corner down, right
print(radolan_grid_ll[0,899, :])

[17.09545175 55.52653305]
[ 4.67593418 46.19287861]
[ 3.09079619 55.53947933]
[15.46817524 46.18345478]


In [45]:
radolan_grid_ll = wrl.georef.get_radolan_grid(900,900, wgs84=True) # [höhe (latitude), breite (longitude)]

listCoordinates = []
listLatitudeComplete = [] # all latitude values for each box in the grid
listLongitudeComplete = [] # all longitude values for each box in the grid
for hoehe in range(900):
    for breite in range(900):
        listLatitudeComplete.append(radolan_grid_ll[hoehe, breite, :][1])
        listLongitudeComplete.append(radolan_grid_ll[hoehe, breite, :][0])
        listCoordinates.append([hoehe, breite])
        

f = open('listLatitudeComplete.pckl', 'wb')
pickle.dump(listLatitudeComplete, f)
f.close()

f = open('listLongitudeComplete.pckl', 'wb')
pickle.dump(listLongitudeComplete, f)
f.close()

f = open('listCoordinates.pckl', 'wb')
pickle.dump(listCoordinates, f)
f.close()

Convert the lists to json format. This files will be used in the App to calculate the Pixel. 

In [46]:
len(listLatitudeComplete)

810000

In [47]:
import json
with open('listLatitudeComplete.json', 'w') as f:
    json.dump(listLatitudeComplete, f)

with open('listLongitudeComplete.json', 'w') as f:
    json.dump(listLongitudeComplete, f)

with open('listCoordinates.json', 'w') as f:
    json.dump(listCoordinates, f)

Check, if it is possible to get the world coordinates from two given Pixels. 

In [14]:
height = 56
width = 457
coordinate = 0; 
for i in range(len(listCoordinates)):
    if(listCoordinates[i][0] == height and listCoordinates[i][1] == width):
        coordinate = [listLatitudeComplete[i], listLongitudeComplete[i]]
        print(coordinate)

[47.66206974368475, 9.172706726253896]


In [4]:
device_position_lat =  47.6606716746533
device_position_long = 9.17242978452521

radolan_grid_ll = wrl.georef.get_radolan_grid(900,900, wgs84=True) # [höhe (latitude), breite (longitude)]
current_pixel_coordinate_lat = 0
current_pixel_coordinate_long = 0

while(True):
    # der Pixel in dem sich der 'Curser' aktuell befindet
    current_coordinate = radolan_grid_ll[current_pixel_coordinate_lat, current_pixel_coordinate_long]
    current_coordinate_lat = current_coordinate[1]
    current_coordinate_long = current_coordinate[0]
    # die Distanz zu den Koordinaten des Gerätes von dem Pixel in dem sich der Cursor aktuell befindet
    current_distance = (current_coordinate_lat - device_position_lat)**2 + (current_coordinate_long - device_position_long)**2
    #print('Current', current_distance)
    
    # nach rechts gehen?
    # die Distanz des rechten nachbarpixels zu den Koordinaten des Gerätes 
    next_coordinate = radolan_grid_ll[current_pixel_coordinate_lat + 1, current_pixel_coordinate_long]
    next_coordinate_lat = next_coordinate[1]
    next_coordinate_long = next_coordinate[0]
    
    next_distance = (next_coordinate_lat - device_position_lat)**2 + (next_coordinate_long - device_position_long)**2
    #print('Next   ', next_distance)
    if next_distance < current_distance:
        current_pixel_coordinate_lat = current_pixel_coordinate_lat + 1
    else:
        # nach oben gehen? 
        # die Distanz des oberen nachbarpixels zu den Koordinaten des Gerätes 
        next_coordinate = radolan_grid_ll[current_pixel_coordinate_lat, current_pixel_coordinate_long + 1]
        next_coordinate_lat = next_coordinate[1]
        next_coordinate_long = next_coordinate[0]
        
        next_distance = (next_coordinate_lat - device_position_lat)**2 + (next_coordinate_long - device_position_long)**2
        
        if next_distance < current_distance:
            current_pixel_coordinate_long = current_pixel_coordinate_long + 1
        
        else: 
            # nach links gehen? 
            # die Distanz des oberen nachbarpixels zu den Koordinaten des Gerätes 
            next_coordinate = radolan_grid_ll[current_pixel_coordinate_lat - 1, current_pixel_coordinate_long]
            next_coordinate_lat = next_coordinate[1]
            next_coordinate_long = next_coordinate[0]

            next_distance = (next_coordinate_lat - device_position_lat)**2 + (next_coordinate_long - device_position_long)**2

            if next_distance < current_distance:
                current_pixel_coordinate_lat = current_pixel_coordinate_lat - 1
            else: 
                # nach unten gehen? 
                # die Distanz des oberen nachbarpixels zu den Koordinaten des Gerätes 
                next_coordinate = radolan_grid_ll[current_pixel_coordinate_lat, current_pixel_coordinate_long - 1]
                next_coordinate_lat = next_coordinate[1]
                next_coordinate_long = next_coordinate[0]

                next_distance = (next_coordinate_lat - device_position_lat)**2 + (next_coordinate_long - device_position_long)**2

                if next_distance < current_distance:
                    current_pixel_coordinate_long = current_pixel_coordinate_long - 1
                else:
                    break
print(current_pixel_coordinate_lat)
print(current_pixel_coordinate_long)

56
457
